# Câu 5: Recommendation - Amazon - Office Products (1.0 điểm)

### If you have more than two laptops/computers or one 32GB-laptop/computer:

- Use the information "reviewerID", "asin" (ProductID), and “overall” (users’ ratings for each product) in dataset ratings_Office_Products.csv to build a model to predict overalls for products that have not been selected by users. Then make recommendations to some users: AZZ56WF4X19G2,A335QXPTV1RIV1, ATIMW8SYGAASW

### If you only have one laptop/computer:

- Use the information "reviewerID", "asin" (ProductID), and “overall” (users’ ratings for each product) in dataset Office_Products_5.json (giải nén tập tin reviews_Office_Products_5.json.gz) to build a model to predict overalls for products that have not been selected by users. Then make recommendations to some users: A3CJ7MHAS9IMAM, A3LGT6UZL99IW1, A21KNRUAA5RK Read more information here: http://jmcauley.ucsd.edu/data/amazon/

In [ ]:
#@title Install Pakages
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark  # Tìm Spark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
#@title Mount to drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/reviews_Office_Products_5.json'

/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/reviews_Office_Products_5.json


In [ ]:
#@title Import Pakages
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col, explode

from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col

# Converting String to index
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [ ]:
import pyspark

In [ ]:
sc = SparkContext()

In [ ]:
spark = SparkSession.builder.appName('Cau5').getOrCreate()

In [ ]:
data = spark.read.json("Office_Products_5.json")

In [ ]:
data.show(5, truncate=True)

+----------+-------+-------+--------------------+-----------+--------------+-------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|       reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+-------------------+--------------------+--------------+
|B00000JBLH| [3, 4]|    5.0|I bought my first...| 09 3, 2004|A32T2H8150OJLU|                ARH|A solid performer...|    1094169600|
|B00000JBLH| [7, 9]|    5.0|WHY THIS BELATED ...|12 15, 2007|A3MAFS04ZABRGO|   Let it Be "Alan"|Price of GOLD is ...|    1197676800|
|B00000JBLH| [3, 3]|    2.0|I have an HP 48GX...| 01 1, 2011|A1F1A0QQP2XVH5|             Mark B|Good functionalit...|    1293840000|
|B00000JBLH| [7, 8]|    5.0|I've started doin...|04 19, 2006| A49R5DBXXQDE5|       R. D Johnson|One of the last o...|    1145404800|
|B00000JBLH| [0, 0]|    5.0|For simple calcul...| 08 4, 2013|A2XRMQA6

In [ ]:
data_sub = data.select(['asin','overall', 'reviewerID'])

In [ ]:
data_sub.count()

53258

In [ ]:
data_sub.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)



In [ ]:
data_sub.show(5, truncate=True)

+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|B00000JBLH|    5.0|A32T2H8150OJLU|
|B00000JBLH|    5.0|A3MAFS04ZABRGO|
|B00000JBLH|    2.0|A1F1A0QQP2XVH5|
|B00000JBLH|    5.0| A49R5DBXXQDE5|
|B00000JBLH|    5.0|A2XRMQA6PJ5ZJ8|
+----------+-------+--------------+
only showing top 5 rows



In [ ]:
# Drop any row that contains missing data
data_sub.na.drop().count()

53258

In [ ]:
data_sub.groupBy("overall").mean().show()

+-------+------------+
|overall|avg(overall)|
+-------+------------+
|    1.0|         1.0|
|    4.0|         4.0|
|    3.0|         3.0|
|    2.0|         2.0|
|    5.0|         5.0|
+-------+------------+



In [ ]:
data_sub.groupBy("overall").count().show()

+-------+-----+
|overall|count|
+-------+-----+
|    1.0| 1130|
|    4.0|15015|
|    3.0| 5060|
|    2.0| 1726|
|    5.0|30327|
+-------+-----+



In [ ]:
data_sub.groupBy("overall").max().show()

+-------+------------+
|overall|max(overall)|
+-------+------------+
|    1.0|         1.0|
|    4.0|         4.0|
|    3.0|         3.0|
|    2.0|         2.0|
|    5.0|         5.0|
+-------+------------+



In [ ]:
data_sub.groupBy("overall").min().show()

+-------+------------+
|overall|min(overall)|
+-------+------------+
|    1.0|         1.0|
|    4.0|         4.0|
|    3.0|         3.0|
|    2.0|         2.0|
|    5.0|         5.0|
+-------+------------+



# Hiểu dữ liệu:
- Dữ liệu không có dữ liệu null
- overall chỉ có 5 giá trị tương ứng với đánh giá 1 đến 5 sao
- Dữ liệu người dùng đánh giá ở 4 và 5 overall tổng thể

In [ ]:
data_sub.select([count(when(col(c).isNull(),c)).alias(c) for c in data_sub.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0


In [ ]:
# Distinct user and movies
user = data_sub.select("reviewerID").distinct().count()
products = data_sub.select("asin").distinct().count()
numerator = data_sub.count()

In [ ]:
display(numerator, user, products)

53258

4905

2420

In [ ]:
# number of rating matrix could contain if no empty cells
denominator = user * products
denominator

11870100

In [ ]:
# Calculating sparsity
sparsity = 1 - (numerator*1.0/denominator)
sparsity

0.9955132644206873

**Chuyển đổi dữ liệu từ string -> int**

In [ ]:
# Create an indexer
indexer = StringIndexer(inputCol='asin', outputCol='asin_idx')
# Indexer indentifies categories in the data
indexer_model = indexer.fit(data_sub)
# Indexer creates a new column with numeric categorical feature
data_indexed = indexer_model.transform(data_sub)
# Repeat the process for the other categorical feature
indexer1 = StringIndexer(inputCol='reviewerID', outputCol='reviewerID_idx')
indexer_model1 = indexer1.fit(data_indexed)
data_indexed= indexer_model1.transform(data_indexed)

In [ ]:
data_indexed.show(5, truncate=True)

+----------+-------+--------------+--------+--------------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|
+----------+-------+--------------+--------+--------------+
|B00000JBLH|    5.0|A32T2H8150OJLU|  1491.0|         289.0|
|B00000JBLH|    5.0|A3MAFS04ZABRGO|  1491.0|        4116.0|
|B00000JBLH|    2.0|A1F1A0QQP2XVH5|  1491.0|         216.0|
|B00000JBLH|    5.0| A49R5DBXXQDE5|  1491.0|        4156.0|
|B00000JBLH|    5.0|A2XRMQA6PJ5ZJ8|  1491.0|        1387.0|
+----------+-------+--------------+--------+--------------+
only showing top 5 rows



In [ ]:
data_indexed.select([count(when(col(c).isNull(),c)).alias(c) for c in data_indexed.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0
asin_idx,0
reviewerID_idx,0


**Xem lại dữ liệu**

In [ ]:
data_check = data_indexed.select(['asin_idx','overall', 'reviewerID_idx'])

In [ ]:
data_check.describe().show()

+-------+-----------------+------------------+------------------+
|summary|         asin_idx|           overall|    reviewerID_idx|
+-------+-----------------+------------------+------------------+
|  count|            53258|             53258|             53258|
|   mean|566.0311878027715| 4.345957414848473|1573.0832738743475|
| stddev|592.2171539786467|0.9302498738235792|1387.3377980258988|
|    min|              0.0|               1.0|               0.0|
|    max|           2419.0|               5.0|            4904.0|
+-------+-----------------+------------------+------------------+



## Tạo train và test set

In [ ]:
# Smaller dataset so we will use 0.8, 0.2
(training, test) = data_indexed.randomSplit([0.8,0.2])

## Áp dụng ALS model

In [ ]:
als = ALS(maxIter=5, regParam=0.09, rank=25, userCol="reviewerID_idx", itemCol="asin_idx", \
          ratingCol="overall", coldStartStrategy="drop", nonnegative=True)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.select(["asin_idx","reviewerID_idx","overall","prediction"]).show(5)

+--------+--------------+-------+----------+
|asin_idx|reviewerID_idx|overall|prediction|
+--------+--------------+-------+----------+
|   148.0|           6.0|    5.0| 4.5100713|
|   148.0|         867.0|    4.0| 4.8820834|
|   148.0|         164.0|    4.0| 3.7906613|
|   148.0|        1335.0|    5.0|  4.373609|
|   148.0|         258.0|    5.0|  4.653311|
+--------+--------------+-------+----------+
only showing top 5 rows



## Đánh giá model

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="overall", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
rmse

1.0526395548866314

In [ ]:
# Providing Recommendations: for all users

In [ ]:
# Get 10 recommendations which have highest rating
user_recs = model.recommendForAllUsers(10)

In [ ]:
for user in user_recs.head(5):
    print(user, "\n")

Row(reviewerID_idx=1580, recommendations=[Row(asin_idx=2288, rating=5.9562554359436035), Row(asin_idx=1785, rating=5.904064655303955), Row(asin_idx=2025, rating=5.8802361488342285), Row(asin_idx=1817, rating=5.862008094787598), Row(asin_idx=1110, rating=5.835598468780518), Row(asin_idx=1683, rating=5.769829750061035), Row(asin_idx=1352, rating=5.7431559562683105), Row(asin_idx=1825, rating=5.735479354858398), Row(asin_idx=2251, rating=5.728366374969482), Row(asin_idx=2131, rating=5.722025394439697)]) 

Row(reviewerID_idx=4900, recommendations=[Row(asin_idx=2025, rating=6.276377201080322), Row(asin_idx=1110, rating=6.256237983703613), Row(asin_idx=1785, rating=6.184288024902344), Row(asin_idx=1817, rating=6.074408054351807), Row(asin_idx=1825, rating=5.876766204833984), Row(asin_idx=2349, rating=5.849661827087402), Row(asin_idx=1475, rating=5.836781978607178), Row(asin_idx=1909, rating=5.818670272827148), Row(asin_idx=1973, rating=5.803783416748047), Row(asin_idx=1683, rating=5.74314212

In [ ]:
# Converting back to string form

In [ ]:
import pandas as pd 
recs=model.recommendForAllUsers(10).toPandas() 
nrecs=recs.recommendations.apply(pd. Series)\
        .merge(recs, right_index = True, left_index = True)\
        .drop(["recommendations"], axis = 1)\
        .melt(id_vars = ['reviewerID_idx'], value_name = "recommendation") \
        .drop("variable", axis = 1) \
        .dropna() 
nrecs=nrecs.sort_values('reviewerID_idx') 
nrecs=pd.concat([nrecs['recommendation'] \
.apply(pd.Series), nrecs['reviewerID_idx']], axis = 1)
nrecs.columns = [ 'ProductID_index', 'Rating', 'UserID_index']

In [ ]:
md = data_indexed.select(['reviewerID', 'reviewerID_idx', 'asin', 'asin_idx']) 
md = md.toPandas() 
dict1 = dict(zip(md['reviewerID_idx'], md['reviewerID'])) 
dict2 = dict(zip(md['asin_idx'],md['asin'])) 
nrecs['reviewerID']=nrecs['UserID_index'].map(dict1) 
nrecs['asin']=nrecs['ProductID_index'].map(dict2) 
nrecs=nrecs.sort_values('reviewerID')
nrecs.reset_index(drop=True, inplace=True) 
new=nrecs[['reviewerID', 'asin', 'Rating']] 
new['recommendations'] = list(zip(new.asin, new.Rating)) 
res=new[['reviewerID', 'recommendations']] 
res_new=res['recommendations'].groupby([res.reviewerID])\
.apply(list).reset_index()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
res_new

,reviewerID,recommendations
0,A00473363TJ8YSZ3YAGG9,"[(B000LPDTV6, 3.7093887329101562), (B001S9HWKS..."
1,A04324702R5O7JRSQNVAH,"[(B003XR480U, 6.029829978942871), (B00006ICSJ,..."
2,A0678500JUN3N7KUG4PS,"[(B004NBZAYS, 5.539435386657715), (B005OQWLXE,..."
3,A100UD67AHFODS,"[(B0030JBTJO, 5.920649528503418), (B00003CYNU,..."
4,A100WO06OQR8BQ,"[(B00003CYNU, 6.4983391761779785), (B002R0DYZ4..."
...,...,...
4898,AZQJ85BTRUQV2,"[(B002R0DX0U, 6.292189121246338), (B002R0DYZ4,..."
4899,AZU2JDR2GVICQ,"[(B002JG10GG, 5.2669243812561035), (B000TD0QU2..."
4900,AZYJE40XW6MFG,"[(B000SBXIA0, 6.53890323638916), (B000HCX7UG, ..."
4901,AZZ5ASC403N74,"[(B003ZSWER6, 7.172185897827148), (B001FNYW9G,..."


In [ ]:
res_new.to_csv("/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Cau_5_Recommendation_Amazon_office_production.csv")

In [ ]:
# In đề xuất cho 3 user A3CJ7MHAS9IMAM, A3LGT6UZL99IW1, A21KNRUAA5RK5E
for i in ['A3CJ7MHAS9IMAM', 'A3LGT6UZL99IW1', 'A21KNRUAA5RK5E']:
    result = res_new.loc[res_new['reviewerID'] == i]['recommendations'].values
    print(' - ReviewerID: %s' %i)
    print(result,'\n')

 - ReviewerID: A3CJ7MHAS9IMAM
[list([('B000SBXIA0', 6.545570373535156), ('B0088VS4Y2', 6.310887336730957), ('B000ANJW1U', 6.553714275360107), ('B002BA5WLE', 6.3081207275390625), ('B001D707CU', 6.685194969177246), ('B001S9HWKS', 6.918809413909912), ('B003ZSWER6', 6.323070526123047), ('B00003CYNU', 6.3933000564575195), ('B002JG10GG', 6.534608364105225), ('B0030JBTJO', 6.581174373626709)])] 

 - ReviewerID: A3LGT6UZL99IW1
[list([('B000LPDTV6', 6.946948528289795), ('B000SBXIA0', 6.806662082672119), ('B002R0DX0U', 6.8423075675964355), ('B002JG10GG', 6.898707866668701), ('B0002T4064', 6.906502723693848), ('B0030A9CHE', 7.042409420013428), ('B002BA5WLE', 6.930212497711182), ('B00003CYNU', 7.06126070022583), ('B001AKX59C', 6.994421482086182), ('B003D7NSPG', 6.8289899826049805)])] 

 - ReviewerID: A21KNRUAA5RK5E
[list([('B000GIOFGQ', 5.676487445831299), ('B003XR480U', 5.6421380043029785), ('B001AKX59C', 5.6617431640625), ('B002JG10GG', 5.939448833465576), ('B00006IFHJ', 5.620556831359863), ('B0